In [1]:
import numpy as np

# Affine 层的实现
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, out):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        return dx

In [2]:
def softmax(a):
    max_a = a - np.max(a)
    exp_a = np.exp(max_a)
    exp_sum = np.sum(exp_a)
    return exp/exp_sum

def cross_entropy(y, t):
    offset = 1e-7
    return -np.sum(t*np.log(y+offset))

# def Relu(x):
#     return np.maximum(0, x)

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y- self.t)/batch_size
        return dx

In [3]:
print(Relu(np.array([]))

SyntaxError: unexpected EOF while parsing (<ipython-input-3-d24641edc0ce>, line 1)

### 神经网络学习的全貌
在进行具体的实现之前，我们再来确认一下神经网络学习的全貌图。神经网络学习的步骤如下所示。
**前提**
神经网络中有合适的权重和偏置，调整权重和偏置以便拟合训练数据的过程称为学习。神经网络的学习分为下面4 个步骤。
1. 步骤1（mini-batch）
从训练数据中随机选择一部分数据。
2. 步骤2（计算梯度）
计算损失函数关于各个权重参数的梯度。
3. 步骤3（更新参数）
将权重参数沿梯度方向进行微小的更新。
4. 步骤4（重复）
重复步骤1、步骤2、步骤3。

In [4]:
import sys, os
sys.path.append('D:\Projects\Python\Deep learning from scratch')
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # initialize weights
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # build layers
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1) # index of max elem
        if t.ndim != 1:
            t = np.argmax(t, axis = 1)
        accuracy = np.sum(y == t)/float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W:self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        self.loss(x, t)
        dout = 1
        dout = self.lastLayer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers: # backward = forward.reverse 
            dout = layer.backward(dout)
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        return grads
        

In [5]:
from dataset.mnist import load_mnist
(x_train, t_train), (x_test,  t_test) = load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(input_size=784, hidden_size = 50, output_size=10)
# x_batch = x_train[:3]
# t_batch = t_train[:3]

# grad_numerical = network.numerical_gradient(x_batch, t_batch)
# grad_backprop = network.gradient(x_batch, t_batch)

# for key in grad_numerical.keys():
#     diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
#     print(key + ":" + str(diff))

In [6]:
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate*grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train accuracy: {:.4f}, test_accuracy: {:.4f}, loss: {:.4f}".format(train_acc, test_acc, loss))

train accuracy: 0.1195, test_accuracy: 0.1178, loss: 2.3015
train accuracy: 0.9056, test_accuracy: 0.9085, loss: 0.4320
train accuracy: 0.9260, test_accuracy: 0.9269, loss: 0.4105
train accuracy: 0.9394, test_accuracy: 0.9364, loss: 0.1994
train accuracy: 0.9484, test_accuracy: 0.9464, loss: 0.3165
train accuracy: 0.9542, test_accuracy: 0.9517, loss: 0.2798
train accuracy: 0.9591, test_accuracy: 0.9526, loss: 0.0927
train accuracy: 0.9632, test_accuracy: 0.9591, loss: 0.0868
train accuracy: 0.9646, test_accuracy: 0.9610, loss: 0.1011
train accuracy: 0.9683, test_accuracy: 0.9619, loss: 0.0530
train accuracy: 0.9704, test_accuracy: 0.9632, loss: 0.0561
train accuracy: 0.9718, test_accuracy: 0.9652, loss: 0.1264
train accuracy: 0.9731, test_accuracy: 0.9664, loss: 0.1496
train accuracy: 0.9757, test_accuracy: 0.9686, loss: 0.0417
train accuracy: 0.9761, test_accuracy: 0.9673, loss: 0.0758
train accuracy: 0.9787, test_accuracy: 0.9704, loss: 0.0407
train accuracy: 0.9784, test_accuracy: 0